In [23]:
import pandas as pd
import utils 
import plotly.graph_objects as go 
import instrument

In [25]:
pair = "EUR_USD"
granularity = "H1"
ma_list=[16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [4]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ["time", "volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [6]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [19]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2022-09-23T03:00:00.000000000Z,0.98281,0.98328,0.98207,0.98236,0.983741,0.988819,-0.005078,NaN,False
1,2022-09-23T04:00:00.000000000Z,0.98238,0.98288,0.98185,0.98230,0.983404,0.988587,-0.005183,-0.005078,False
2,2022-09-23T05:00:00.000000000Z,0.98230,0.98332,0.98224,0.98295,0.983350,0.988349,-0.004999,-0.005183,False
3,2022-09-23T06:00:00.000000000Z,0.98294,0.98382,0.98084,0.98125,0.983333,0.988065,-0.004731,-0.004999,False
4,2022-09-23T07:00:00.000000000Z,0.98124,0.98140,0.97672,0.97756,0.982931,0.987724,-0.004793,-0.004731,False


In [12]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 
    return 0

In [13]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [15]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
89,2022-09-28T20:00:00.000000000Z,0.97324,0.97386,0.97290,0.97368,0.962882,0.962101,0.000781,-0.000213,True
214,2022-10-06T01:00:00.000000000Z,0.99168,0.99193,0.99030,0.99110,0.989115,0.989145,-0.000030,0.000322,True
315,2022-10-12T06:00:00.000000000Z,0.97210,0.97330,0.97152,0.97215,0.971659,0.971571,0.000089,-0.000099,True
316,2022-10-12T07:00:00.000000000Z,0.97216,0.97264,0.96961,0.97074,0.971354,0.971451,-0.000097,0.000089,True
344,2022-10-13T11:00:00.000000000Z,0.97250,0.97533,0.97210,0.97450,0.970936,0.970730,0.000206,-0.000001,True


In [20]:
df_plot = df_ma.iloc[50:100].copy()

In [21]:
df_plot.shape

(50, 10)

In [22]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A068",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
)) 
for ma in ma_list:
    col=f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
                             y=df_plot[col], 
                            line=dict(width=2),
                            line_shape='spline', 
                            name=col))
fig.update_layout(width=900, height=400, 
                  margin=dict(l=10, r=10, b=10, t=10),
                  font=dict(size=10, color="#e1e1e1"),
                  paper_bgcolor="#1e1e1e",
                  plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True, fixedrange=True, rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()